In [5]:
#detecting fraud without encoding in BM
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=LAPTOP-CTTVUGH7;"
            "Database=tables_fait_1;"
            "Trusted_Connection=yes;")

cnxn = pyodbc.connect(cnxn_str)

In [7]:
cursor = cnxn.cursor()

In [8]:
import sqlalchemy
engine = sqlalchemy.create_engine("mssql+pyodbc://LAPTOP-CTTVUGH7/tables_fait_1?driver=SQL Server?Trusted_Connection=yes")
conn = engine.connect()

In [9]:
#improtation fait BM
fait_BM = pd.read_sql("SELECT * FROM [fait_BM]", cnxn)

In [10]:
fait_BM = fait_BM.drop(['index'], axis=1)

In [11]:
#duplication des données selon CodeBonusMalus_id
fait_BM.duplicated(subset=['CodeBonusMalus_id']).sum()

0


1)policy type is "individual" but the client has more than 3 vehicules

In [12]:
query = """SELECT count(CodeVehicule_id) as count,CodeAssure_id
        FROM fait_BM where typePolice='I' 
        group by CodeAssure_id having count(CodeVehicule_id) >=3 """
results = cursor.execute(query).fetchall()

In [13]:
#sql to dataframe
dataframe = pd.read_sql(query, con = conn)
dataframe

,count,CodeAssure_id
0,4,220752
1,3,104464
2,3,581997
3,3,696506
4,3,233394
...,...,...
36117,3,215140
36118,4,361664
36119,5,966321
36120,3,169584


In [14]:
condition=(fait_BM["CodeAssure_id"].isin(dataframe["CodeAssure_id"]))
fait_BM.loc[condition,'Fraud']=1


2)policy type is "fleet " but the client has less than 3 cars

In [15]:
query = """SELECT count(CodeVehicule_id) as count,CodeAssure_id
        FROM fait_BM where typePolice='F' 
    group by CodeAssure_id having count(CodeVehicule_id) <3 """
results = cursor.execute(query).fetchall()



In [16]:
dataframe = pd.read_sql(query, con = conn)
dataframe

,count,CodeAssure_id
0,1,329331
1,1,1063693
2,1,1072206
3,1,1072467
4,2,1072567
...,...,...
1834,1,1065719
1835,1,1072503
1836,1,99052
1837,1,276155


In [17]:
condition=(fait_BM["CodeAssure_id"].isin(dataframe["CodeAssure_id"]))
fait_BM.loc[condition,'Fraud']=2


3) usage code=1 (private use) so policy type must not be "fleet"

In [18]:
query = "SELECT CodeVehicule_id FROM fait_BM where codeUsage=1 and typePolice='F' "
results = cursor.execute(query).fetchall()

In [19]:
dataframe = pd.read_sql(query, con = conn)
dataframe

,CodeVehicule_id
0,32130
1,32132
2,32133
3,32134
4,32135
...,...
4125,1030192
4126,1089138
4127,334198
4128,1089139


In [20]:
condition=(fait_BM["CodeVehicule_id"].isin(dataframe["CodeVehicule_id"]))
fait_BM.loc[condition,'Fraud']=3


4) usage code=14 or 15 (rent or travel agency) so policy type must not be "individual"

In [21]:
query = "SELECT CodeVehicule_id FROM fait_BM where codeUsage in (15,14) and typePolice='I' "
results = cursor.execute(query).fetchall()

In [22]:
dataframe = pd.read_sql(query, con = conn)
dataframe

,CodeVehicule_id
0,48405
1,31730
2,31733
3,52239
4,52318
...,...
2349,490191
2350,1070565
2351,1070741
2352,1071173


In [23]:
condition=(fait_BM["CodeVehicule_id"].isin(dataframe["CodeVehicule_id"]))
fait_BM.loc[condition,'Fraud']=4



5) client has more than 4 vehicules so policy type must not be "individual"

In [24]:
query = """SELECT count (CodeVehicule_id) as count_CodeVehicule_id,CodeAssure_id 
        FROM fait_BM where typePolice='I' 
        group by CodeAssure_id having (count (CodeVehicule_id)) >=3 """
results = cursor.execute(query).fetchall()

In [25]:
dataframe = pd.read_sql(query, con = conn)
dataframe

,count_CodeVehicule_id,CodeAssure_id
0,6,492037
1,3,581997
2,3,491205
3,3,878207
4,3,97657
...,...,...
36117,3,907325
36118,3,716340
36119,3,222154
36120,4,449744


In [26]:
condition=(fait_BM["CodeAssure_id"].isin(dataframe["CodeAssure_id"]))
fait_BM.loc[condition,'Fraud']=5

6) multi insurance fraud

In [27]:
query = """SELECT CodeVehicule_id ,count (codeCompagnie) as count_codeCompagnie 
        FROM fait_BM where typePolice='I' group by CodeVehicule_id having (count (codeCompagnie)) >=3 """
results = cursor.execute(query).fetchall()

In [28]:
dataframe = pd.read_sql(query, con = conn)
dataframe

,CodeVehicule_id,count_codeCompagnie
0,87321,3
1,134529,3
2,164106,3
3,562568,3
4,179663,3
...,...,...
23285,150516,3
23286,147940,3
23287,656065,3
23288,81564,3


In [29]:
condition=(fait_BM["CodeVehicule_id"].isin(dataframe["CodeVehicule_id"]))
fait_BM.loc[condition,'Fraud']=6


7) wrecked vehicule still has an operational policy

In [30]:
query = "SELECT CodeVehicule_id FROM fait_BM where ETAT_VEHICULE='R' "

In [31]:
dataframe = pd.read_sql(query, con = conn)
dataframe

,CodeVehicule_id
0,68442
1,38922
2,68443
3,68444
4,68445
...,...
1523,1089135
1524,1089136
1525,1089137
1526,1030196


In [32]:
condition=(fait_BM["CodeVehicule_id"].isin(dataframe["CodeVehicule_id"]))
fait_BM.loc[condition,'Fraud']=7

8)unusual date

In [33]:
query = "SELECT DATE_AFFECTATION FROM [fait_BM] where DATE_AFFECTATION>'2021-03-16 00:00:00'"
results = cursor.execute(query).fetchall()

In [34]:
dataframe = pd.read_sql(query, con = conn)
dataframe 

,DATE_AFFECTATION
0,2516-07-08 00:00:00
1,2115-03-02 00:00:00


In [35]:
condition=(fait_BM["DATE_AFFECTATION"].isin(dataframe["DATE_AFFECTATION"]))
fait_BM.loc[condition,'Fraud']=8

filling missing values

In [36]:
fait_BM['energie']=fait_BM['energie'].replace(['0'],'unkown')

adding num accidents

In [37]:
query = """SELECT CodeAssure_id,count(CodeSinistre_id) as num_accidents FROM fait_Sinistre_fraud where
        pourcentadeDeResponsabilite>50 And Fraud=0 group by CodeAssure_id"""
results = cursor.execute(query).fetchall()
num_accidents = pd.read_sql(query, con = conn)
num_accidents.CodeAssure_id = num_accidents.CodeAssure_id.astype(int)
num_accidents

,CodeAssure_id,num_accidents
0,66491,1
1,337793,1
2,1642059,1
3,1056048,1
4,470774,1
...,...,...
190774,975529,1
190775,1065946,1
190776,1564118,1
190777,799811,1


In [38]:
fait_BM = pd.merge(fait_BM,
                 num_accidents,
                 on='CodeAssure_id', 
                 how='left')

In [39]:
fait_BM['num_accidents'] = fait_BM['num_accidents'].fillna(0)

saving to ssms

In [40]:
import sqlalchemy
engine = sqlalchemy.create_engine("mssql+pyodbc://LAPTOP-CTTVUGH7/tables_fait_1?driver=SQL Server?Trusted_Connection=yes")
conn = engine.connect()

In [41]:
fait_BM.to_sql('fait_BM_fraud', conn, if_exists='replace')